# Ordinary Least Squares
## Course recap
This lab consists in implementing the **Ordinary Least Squares** (OLS) algorithm, which is **a linear regression with a least-squares penalty**. Given a training set $ D = \left\{ \left(x^{(i)}, y^{(i)}\right), x^{(i)} \in \mathcal{X}, y^{(i)} \in \mathcal{Y}, i \in \{1, \dots, n \}  \right\}$, recall (from lectures 1 and 2) OLS aims at minimizing the following cost function $J$:
$$J(\theta) = \dfrac{1}{2} \sum_{i = 1}^{n} \left( h\left(x^{(i)}\right) - y^{(i)} \right)^2$$
where 
$$h(x) = \sum_{j = 0}^{d} \theta_j x_j = \theta^T x.$$

For the sake of simplicity, we will be working on a small training set (the one we used in lectures 1 and 2) $D$:

| living area (m$^2$) | price (1000's BGN)|
|--------------------:|------------------:|
|                 50  |         30        |
|                 76  |         48        |
|                 26  |         12        |
|                102  |         90        |

## Defining the training set
**Exercise 1**: Define variables that will contain the training set.

**Note**: Do not forget the intercept!

## Prediction function
**Exercise**: Define a function `predict` that takes as parameter *the feature vector* $x$ and *the model* $\theta$ and returns the predicted label:
$$ \hat{y} = h(x) = \theta^T x = \sum_{j = 0}^d \theta_j x_j$$

In [3]:
def predict(x, theta):
    ## TODO
    return 0

## Defining the cost function
**Exercise**: Define a function `cost_function` that takes as parameter *the predicted label* $y$ and *the actual label* $\hat{y}$ of a single sample and returns the loss of this pair given by:
$$ \ell \left( y - \hat{y} \right) = \left( y - \hat{y} \right)^2$$

In [4]:
def cost_function(y, y_hat):
    ## TODO
    return 0

## Defining the gradient of the cost function
**Exercise**: Define a function `gradient_cost_function`.

In [6]:
def gradient(x, theta, y):
    #TODO
    return x